In [1]:
import joblib
import pandas as pd

from rdkit import Chem
from sklearn.metrics import roc_auc_score
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool, global_add_pool
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn.functional as F

import numpy as np
from rdkit.Chem import rdFingerprintGenerator
import deepchem as dc
import random
import optuna

from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    log_loss,
    roc_auc_score,
    precision_score,
    recall_score,
    confusion_matrix,
)

c:\Users\predator\anaconda3\envs\dili-3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'dgl'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'


In [2]:
# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


# Load pre-trained models

In [3]:
# load the results back
loaded_results = joblib.load("models/tpot_results_x20.joblib")

# Convert results to DataFrame
tpot_df = pd.DataFrame(loaded_results)
tpot_df.head()

,Best model,PCA Components,Model Name,Parameters,AUC,Precision,Recall,Sensitivity,Specificity
0,"((ExtraTreeClassifier(criterion='entropy', max...",10,ExtraTreesClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.848730,0.901478,0.828054,0.828054,0.692308
1,"((ExtraTreeClassifier(max_features=0.8, min_sa...",20,ExtraTreesClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.877271,0.929648,0.837104,0.837104,0.784615
2,"((ExtraTreeClassifier(criterion='entropy', max...",50,ExtraTreesClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.880752,0.888412,0.936652,0.936652,0.600000
3,"(XGBClassifier(base_score=None, booster=None, ...",100,XGBClassifier,"{'objective': 'binary:logistic', 'base_score':...",0.873234,0.903670,0.891403,0.891403,0.676923


In [4]:
model1 = loaded_results[1]["Best model"]
model1

Pipeline(steps=[('extratreesclassifier',
                 ExtraTreesClassifier(bootstrap=True, max_features=0.8,
                                      min_samples_leaf=19, min_samples_split=5,
                                      random_state=42))])

In [6]:
class GCN(torch.nn.Module):
    def __init__(
        self,
        num_node_features,
        num_classes,
        num_layers=3,
        hidden_dim=64,
        dropout_prob=0.5,
        activation="relu",
    ):
        super(GCN, self).__init__()

        # Store activation function dynamically
        if activation == "relu":
            self.activation = F.relu
        elif activation == "tanh":
            self.activation = F.tanh
        else:
            raise ValueError("Unsupported activation function")

        self.dropout_prob = dropout_prob

        # Dynamically define the GCN layers
        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(num_node_features, hidden_dim))
        for _ in range(num_layers - 1):
            self.convs.append(GCNConv(hidden_dim, hidden_dim))

        # Final fully connected layer
        self.fc = torch.nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # Apply GCN layers dynamically
        for conv in self.convs:
            x = conv(x, edge_index)
            x = self.activation(x)

        # Global pooling (combine different pooling methods)
        x = torch.cat([global_mean_pool(x, batch), global_add_pool(x, batch)], dim=1)

        # Apply dropout
        x = F.dropout(x, p=self.dropout_prob, training=self.training)

        # Final classification layer
        return F.log_softmax(self.fc(x), dim=1)


# load model from pth
# best_params = {
#     "hidden_dim": 138,
#     "dropout_prob": 0.3051102112946549,
#     "lr": 0.0007729766578598647,
#     "weight_decay": 1.0482423771632127e-05,
#     "num_layers": 4,
#     "activation": "relu",
# }

best_params = {
    "hidden_dim": 210,
    "dropout_prob": 0.4169127419744815,
    "lr": 0.0007529508911659432,
    "weight_decay": 9.903281752862725e-06,
    "num_layers": 5,
    "activation": "relu",
}


model2 = GCN(
    num_node_features=70,
    num_classes=2,
    num_layers=best_params["num_layers"],
    hidden_dim=best_params["hidden_dim"],
    dropout_prob=best_params["dropout_prob"],
    activation=best_params["activation"],
).to(device)

model2.load_state_dict(torch.load("models/graph_gcn_x20_filtered_outliers.pth"))

C:\Users\predator\AppData\Local\Temp\ipykernel_22588\3123032249.py:79: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model2.load_state_dict(torch.load("models/graph_gcn_x20_

<All keys matched successfully>

In [8]:
# load the results back
loaded_results = joblib.load("models/tpot_transformer_results_x20.joblib")

# Convert results to DataFrame
tpot_df = pd.DataFrame(loaded_results)
tpot_df.head()

,Best model,PCA Components,Model Name,Parameters,AUC,Precision,Recall,Sensitivity,Specificity
0,"(XGBClassifier(base_score=None, booster=None, ...",10,XGBClassifier,"{'objective': 'binary:logistic', 'base_score':...",0.647128,0.804598,0.950226,0.950226,0.215385
1,"((ExtraTreeClassifier(criterion='entropy', max...",20,ExtraTreesClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.616464,0.793893,0.941176,0.941176,0.169231
2,"(MaxAbsScaler(), (ExtraTreeClassifier(criterio...",50,ExtraTreesClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.591472,0.785992,0.914027,0.914027,0.153846
3,"(XGBClassifier(base_score=None, booster=None, ...",100,XGBClassifier,"{'objective': 'binary:logistic', 'base_score':...",0.594466,0.796935,0.941176,0.941176,0.184615
4,"(ZeroCount(), (ExtraTreeClassifier(criterion='...",200,ExtraTreesClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_...",0.595580,0.790514,0.904977,0.904977,0.184615


In [9]:
model3 = loaded_results[0]["Best model"]
model3

Pipeline(steps=[('xgbclassifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.01,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=6, max_leaves=None, min_child_weight=4,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=100, n_jobs=1,
                               num_parallel_tree=None, random_state=42, ...))])

# Data preparation

## PCA prep

In [10]:
# Load pd_train
pd_train = pd.read_parquet("data/training_class_x20.parquet")
pd_train["label"] = pd_train["Liver"].apply(lambda x: 1 if x == "Hepatotoxicity" else 0)
print(pd_train.shape)

(24759, 16096)


In [11]:
# Load pd_test
pd_test = pd.read_parquet("data/testing_class_x20.parquet")
pd_test["label"] = pd_test["Liver"].apply(lambda x: 1 if x == "Hepatotoxicity" else 0)
print(pd_test.shape)

(286, 16096)


In [12]:
X_train = pd_train.drop(columns=["Liver", "label", "Name", "Smiles"])
y_train = pd_train["label"]

X_test = pd_test.drop(columns=["Liver", "label", "Name", "Smiles"])
y_test = pd_test["label"]

In [20]:
# get X PCA components 20
pca = PCA(n_components=20)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

## Graph prep

In [14]:
# Featurization using DeepChem's MolGraphConvFeaturizer
from utils.SmilesEnumeration import SmilesEnumerator


def featurize_smiles(smiles):
    featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
    graph_data = featurizer.featurize([smiles])[0]

    # Get DeepChem atom features
    atom_features_deepchem = graph_data.node_features

    return atom_features_deepchem


# Function to generate Morgan Fingerprints (ECFP)
def generate_ecfp(smiles):
    # Morgan fingerprint generator
    mfgen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=4096)

    molecule = Chem.MolFromSmiles(smiles)
    if molecule is None:
        return None
    return mfgen.GetFingerprintAsNumPy(molecule)


# Function to convert SMILES to PyTorch Geometric Data object using DeepChem featurizer
def smiles_to_graph_featurizer(smiles):
    # Featurization using DeepChem
    featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)

    # Featurize the SMILES string using DeepChem
    graph_data = featurizer.featurize([smiles])[0]
    return graph_data.node_features, graph_data.edge_features, graph_data.edge_index


# Function to extract atom features
def atom_features(atom, ecfp):
    # Get the atom index for corresponding ECFP value
    atom_idx = atom.GetIdx()

    return [
        atom.GetAtomicNum(),  # Atomic number
        atom.GetDegree(),  # Number of bonds
        atom.GetTotalNumHs(),  # Total number of hydrogens
        atom.GetFormalCharge(),  # Formal charge of the atom
        atom.GetImplicitValence(),  # Implicit valence
        atom.GetNumRadicalElectrons(),  # Number of radical electrons
        int(atom.GetIsAromatic()),  # Is the atom aromatic?
        atom.GetMass(),  # Atomic mass
        atom.GetHybridization().real,  # Hybridization state (SP, SP2, SP3, etc.)
        ecfp[atom_idx],  # Morgan fingerprint (ECFP) for the atom
    ]


# Function to extract bond features
def bond_features(bond):
    bond_type = bond.GetBondTypeAsDouble()  # Bond type as a float
    is_aromatic = bond.GetIsAromatic()  # Aromatic bond
    is_conjugated = bond.GetIsConjugated()  # Conjugated bond
    is_in_ring = bond.IsInRing()  # Whether the bond is part of a ring
    stereo = bond.GetStereo()  # Bond stereochemistry

    # Convert stereo information to a one-hot encoded format
    stereo_one_hot = [0, 0, 0, 0]  # Stereo options: None, E, Z, Other
    if stereo == Chem.BondStereo.STEREONONE:
        stereo_one_hot[0] = 1
    elif stereo == Chem.BondStereo.STEREOE:
        stereo_one_hot[1] = 1
    elif stereo == Chem.BondStereo.STEREOZ:
        stereo_one_hot[2] = 1
    else:
        stereo_one_hot[3] = 1

    # Combine all features into a single tensor
    return [
        bond_type,
        float(is_aromatic),
        float(is_conjugated),
        float(is_in_ring),
    ] + stereo_one_hot


# Convert SMILES to PyTorch Geometric Data object
def smiles_to_graph(smiles, label):
    mol = Chem.MolFromSmiles(smiles)

    atom_features_list = []
    edge_index = []
    edge_attr = []

    # DeepChem features
    atom_features_deepchem = featurize_smiles(smiles)

    # Generate Morgan Fingerprint (ECFP)
    ecfp_features = generate_ecfp(smiles)

    # Generate Molecule Graph Convolution features
    mol_graph_node_features, mol_graph_edge_features, mol_graph_edge_index = (
        smiles_to_graph_featurizer(smiles)
    )

    # Nodes (atoms)
    for atom in mol.GetAtoms():
        atom_features_list.append(atom_features(atom, ecfp_features))

    atom_features_list = np.array(atom_features_list)

    # Edges (bonds)
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()

        # Append bidirectional edges for undirected graphs
        edge_index.append([i, j])
        edge_index.append([j, i])

        # Append bond features for both directions
        edge_attr.append(bond_features(bond))
        edge_attr.append(bond_features(bond))

    # Convert atom features to a tensor
    combined_features = np.concatenate(
        (atom_features_list, atom_features_deepchem, mol_graph_node_features), axis=1
    )
    x = torch.tensor(combined_features, dtype=torch.float)

    # Convert edge indices and features to tensors, handle empty edge case
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

    # combine edge features from ECFP and MolGraphConv
    edge_attr = np.array(edge_attr)
    edge_attr = np.concatenate((edge_attr, mol_graph_edge_features), axis=1)
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)

    # Label (target)
    y = torch.tensor([label], dtype=torch.long)

    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)


# Function to load data from CSV and apply SMILES augmentation for training
def load_data_from_csv(file_path, augment=False, augment_times=1):
    df = pd.read_csv(file_path)

    # if "Train" in file_path:
    #     print("Filter outliers")
    #     # calculate the length of the Smiles strings
    #     df["smiles_len"] = df["Smiles"].apply(lambda x: len(x))
    #     df = df[df["smiles_len"] <= df["smiles_len"].quantile(0.95)]

    smiles_list = df["Smiles"].values
    labels = df["Liver"].apply(lambda x: 1 if x == "Hepatotoxicity" else 0).values

    data_list = []

    # Initialize the SmilesEnumerator for data augmentation
    for smiles, label in zip(smiles_list, labels):
        # For test data, no augmentation, just use canonical SMILES
        graph_data = smiles_to_graph(smiles, label)
        data_list.append(graph_data)

    data_list = []

    # Initialize the SmilesEnumerator for data augmentation
    sme = SmilesEnumerator()
    for smiles, label in zip(smiles_list, labels):
        if augment:
            # Apply SMILES randomization (augmentation) for training data
            for _ in range(augment_times):
                augmented_smiles = sme.randomize_smiles(smiles)
                graph_data = smiles_to_graph(augmented_smiles, label)
                data_list.append(graph_data)
        else:
            # For test data, no augmentation, just use canonical SMILES
            graph_data = smiles_to_graph(smiles, label)
            data_list.append(graph_data)

    return data_list


# Function to load data from parquet and apply SMILES augmentation for training
def load_data_from_parquet(file_path):
    df = pd.read_parquet(file_path)

    smiles_list = df["Smiles"].values
    labels = df["Liver"].apply(lambda x: 1 if x == "Hepatotoxicity" else 0).values

    data_list = []

    # Initialize the SmilesEnumerator for data augmentation
    for smiles, label in zip(smiles_list, labels):
        # For test data, no augmentation, just use canonical SMILES
        graph_data = smiles_to_graph(smiles, label)
        data_list.append(graph_data)

    return data_list

In [15]:
# Load training and testing data
# training_data = load_data_from_csv("data_smiles/Training_Group.csv")
# testing_data = load_data_from_csv("data_smiles/Testing_Group.csv")

# Load training and testing data
training_data = load_data_from_parquet("data/training_class_x20.parquet")
testing_data = load_data_from_parquet("data/testing_class_x20.parquet")

In [16]:
# Create data loaders
train_loader = DataLoader(training_data, batch_size=32, shuffle=True)
test_loader = DataLoader(testing_data, batch_size=32, shuffle=False)

## Transfomer prep

In [17]:
# get X PCA components 10
pca = PCA(n_components=10)
X_train_transformer_pca = pca.fit_transform(X_train)
X_test_transformer_pca = pca.transform(X_test)

# Model training

In [21]:
tpot_preds = model1.predict_proba(X_train_pca)[:, 1]
tpot_transformer_preds = model3.predict_proba(X_train_transformer_pca)[:, 1]

# Predict on validation set
model2.eval()
gcn_preds = []
with torch.no_grad():
    for data in train_loader:
        data = data.to(device)
        out = model2(data)
        gcn_preds.extend(out[:, 1].cpu().numpy())  # Probability for class 1

gcn_preds = np.array(gcn_preds)

# Combine predictions as stacked features
stacked_features_train = np.vstack([tpot_preds, tpot_transformer_preds, gcn_preds]).T
stacked_features_train.shape

(24759, 3)

In [22]:
# Train meta-model
meta_model = LogisticRegression(random_state=42)
meta_model.fit(stacked_features_train, y_train)

LogisticRegression(random_state=42)

In [24]:
from sklearn.metrics import roc_auc_score, accuracy_score

# TPOT predictions on test set
tpot_test_preds = model1.predict_proba(X_test_pca)[:, 1]
tpot_test_transformer_preds = model3.predict_proba(X_test_transformer_pca)[:, 1]

# GCN predictions on test set
gcn_test_preds = []
with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        out = model2(data)
        gcn_test_preds.extend(out[:, 1].cpu().numpy())  # Probability for class 1

# Stack test predictions
stacked_features_test = np.vstack([tpot_test_preds, tpot_test_transformer_preds, gcn_test_preds]).T

# Make final predictions with meta-model
final_predictions = meta_model.predict(stacked_features_test)
final_auc = roc_auc_score(y_test, final_predictions)
final_accuracy = accuracy_score(y_test, final_predictions)

print("Stacked Model Test AUC:", final_auc)
print("Stacked Model Test Accuracy:", final_accuracy)

Stacked Model Test AUC: 0.809502262443439
Stacked Model Test Accuracy: 0.8146853146853147


In [26]:
from tpot import TPOTClassifier

# Set up TPOT with basic configuration
tpot = TPOTClassifier(
    generations=5,
    population_size=20,
    verbosity=2,
    scoring="roc_auc",  # AUC
    random_state=42,
    n_jobs=16,  # Use 4 cores
)

# Fit TPOT on the PCA-transformed data
tpot.fit(stacked_features_train, y_train)

# Access the best pipeline directly after fitting
best_pipeline = tpot.fitted_pipeline_

                                                                             
Generation 1 - Current best internal CV score: 0.9700563196367572
                                                                             
Generation 2 - Current best internal CV score: 0.9701412431153719
                                                                             
Generation 3 - Current best internal CV score: 0.9701412431153719
                                                                             
Generation 4 - Current best internal CV score: 0.9701874303007276
                                                                              
Generation 5 - Current best internal CV score: 0.9701874303007276
                                                                              
Best pipeline: LogisticRegression(GaussianNB(input_matrix), C=5.0, dual=False, penalty=l2)


In [27]:
# Check if the model has predict_proba
from pprint import pprint


if hasattr(best_pipeline, "predict_proba"):
    y_pred = best_pipeline.predict_proba(stacked_features_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred)
else:
    # Fallback to accuracy if predict_proba is unavailable
    y_pred = best_pipeline.predict(stacked_features_test)
    auc = accuracy_score(
        y_test, y_pred
    )  # Substitute with accuracy if AUC can't be calculated

y_pred = (y_pred > 0.5).astype(int)  # Binarize predictions for threshold of 0.5
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)  # Same as sensitivity for positive class

# Calculate sensitivity and specificity
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

# Extract model name and parameters
model_name = type(best_pipeline.steps[-1][1]).__name__  # Name of the final estimator
model_params = best_pipeline.steps[-1][1].get_params()  # Params of the final estimator

# Append results to list
result = {
    "Best model": best_pipeline,
    "Model Name": model_name,
    "Parameters": model_params,
    "AUC": auc,
    "Precision": precision,
    "Recall": recall,
    "Sensitivity": sensitivity,
    "Specificity": specificity,
}

pprint(result)

{'AUC': 0.8748346675948486,
 'Best model': Pipeline(steps=[('stackingestimator',
                 StackingEstimator(estimator=GaussianNB())),
                ('logisticregression',
                 LogisticRegression(C=5.0, random_state=42))]),
 'Model Name': 'LogisticRegression',
 'Parameters': {'C': 5.0,
                'class_weight': None,
                'dual': False,
                'fit_intercept': True,
                'intercept_scaling': 1,
                'l1_ratio': None,
                'max_iter': 100,
                'multi_class': 'deprecated',
                'n_jobs': None,
                'penalty': 'l2',
                'random_state': 42,
                'solver': 'lbfgs',
                'tol': 0.0001,
                'verbose': 0,
                'warm_start': False},
 'Precision': 0.9336734693877551,
 'Recall': 0.8280542986425339,
 'Sensitivity': 0.8280542986425339,
 'Specificity': 0.8}


# Optuna

In [28]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


def compute_loss(y_hat, y):
    y_hat = np.clip(y_hat, 1e-7, 1 - 1e-7)

    return (-y * np.log(y_hat) - (1 - y) * np.log(1 - y_hat)).mean()


def predict(X, theta):
    dot_product = np.dot(X, theta)
    y_hat = sigmoid(dot_product)

    return y_hat


def compute_gradient(X, y, y_hat):
    return np.dot(X.T, (y_hat - y)) / y.size


def update_theta(theta, gradient, lr):
    return theta - lr * gradient


def compute_accuracy(X, y, theta):
    y_hat = predict(X, theta).round()
    acc = (y_hat == y).mean()

    return acc

In [29]:
# Define inference function for each model
def get_model_predictions(tpot_model, tpot_transformer_model, gcn_model, X_pca, X_transformer_pca, loader, device):
    # TPOT model predictions
    tpot_preds = tpot_model.predict_proba(X_pca)[:, 1]
    tpot_transformer_preds = tpot_transformer_model.predict_proba(X_transformer_pca)[:, 1]

    # GCN model predictions
    gcn_preds = []
    gcn_model.eval()
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = gcn_model(data)
            gcn_preds.extend(out[:, 1].cpu().numpy())  # Probability for class 1
    gcn_preds = np.array(gcn_preds)

    # Combine predictions into stacked feature set
    stacked_features = np.vstack([tpot_preds, tpot_transformer_preds, gcn_preds]).T
    return stacked_features

In [30]:
# Optuna objective function to optimize the final stacking ensemble
def objective(trial):
    # Hyperparameter suggestions for stacking model
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)
    max_iter = trial.suggest_int("max_iter", 50, 200)

    # Prepare stacked training features and target
    stacked_train = get_model_predictions(
        model1, model3, model2, X_train_pca, X_train_transformer_pca, train_loader, device
    )

    # Initialize stacking model (Logistic Regression)
    stacking_model = LogisticRegression(
        C=lr, max_iter=max_iter, random_state=42
    )

    # Fit stacking model on stacked features
    stacking_model.fit(stacked_train, y_train)

    # Get validation stacked features for AUC evaluation
    stacked_val = get_model_predictions(
        model1, model3, model2, X_test_pca, X_test_transformer_pca, test_loader, device
    )
    val_preds = stacking_model.predict_proba(stacked_val)[:, 1]
    val_auc = roc_auc_score(y_test, val_preds)

    # Report AUC for Optuna and check for pruning
    trial.report(val_auc, step=0)
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

    return val_auc

In [31]:
# Run Optuna study to find the best parameters for the stacking model
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

[I 2024-11-14 09:40:56,606] A new study created in memory with name: no-name-d44c905a-1d2f-4988-9fb7-a46d96dc1029
C:\Users\predator\AppData\Local\Temp\ipykernel_22588\823997813.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)
[I 2024-11-14 09:41:01,763] Trial 0 finished with value: 0.8765053950574312 and parameters: {'lr': 0.0007286739191332247, 'max_iter': 62}. Best is trial 0 with value: 0.8765053950574312.
C:\Users\predator\AppData\Local\Temp\ipykernel_22588\823997813.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)
[I 2024-11-14 09:41:06,449] Trial 1 finished with v

In [32]:
# Best trial results
print("Best trial:")
print(f"AUC Score: {study.best_trial.value}")
print("Best Parameters:")
for key, value in study.best_trial.params.items():
    print(f"  {key}: {value}")

Best trial:
AUC Score: 0.893699965193178
Best Parameters:
  lr: 2.0688961623689588e-05
  max_iter: 61


In [34]:
from sklearn.metrics import f1_score


def evaluate_with_optimal_threshold(y_true, y_pred_proba):
    # Calculate AUC
    auc = roc_auc_score(y_true, y_pred_proba)

    # Initialize variables to store the best threshold and performance metrics
    best_threshold = 0.5
    best_metrics = {
        "accuracy": 0,
        "precision": 0,
        "recall": 0,
        "sensitivity": 0,
        "specificity": 0,
        "f1": 0,
    }

    # Iterate over possible thresholds
    for threshold in np.arange(0.0, 1.0, 0.01):
        # Binarize predictions based on the current threshold
        y_pred = (y_pred_proba >= threshold).astype(int)

        # Calculate precision, recall, accuracy, sensitivity, and specificity
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)  # Same as sensitivity for positive class
        accuracy = accuracy_score(y_true, y_pred)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        f1 = f1_score(y_true, y_pred)

        # Update the best threshold if F1 score is higher than previous best
        if f1 > best_metrics["f1"]:
            best_threshold = threshold
            best_metrics = {
                "accuracy": accuracy,
                "precision": precision,
                "recall": recall,
                "sensitivity": sensitivity,
                "specificity": specificity,
                "f1": f1,
            }

    # Compile all results
    result = {
        "Best Threshold": best_threshold,
        "AUC": auc,
        **best_metrics,
    }
    return result

In [35]:
from sklearn.metrics import roc_auc_score, accuracy_score

# TPOT predictions on test set
tpot_test_preds = model1.predict_proba(X_test_pca)[:, 1]
tpot_test_transformer_preds = model3.predict_proba(X_test_transformer_pca)[:, 1]

# GCN predictions on test set
gcn_test_preds = []
with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        out = model2(data)
        gcn_test_preds.extend(out[:, 1].cpu().numpy())  # Probability for class 1

# Stack test predictions
stacked_features_test = np.vstack([tpot_test_preds, tpot_test_transformer_preds, gcn_test_preds]).T

# Make final predictions with meta-model
final_predictions = meta_model.predict(stacked_features_test)

# Evaluate metrics
result = evaluate_with_optimal_threshold(y_test, final_predictions)
pprint(result)

{'AUC': 0.809502262443439,
 'Best Threshold': 0.01,
 'accuracy': 0.8146853146853147,
 'f1': 0.8722891566265061,
 'precision': 0.9329896907216495,
 'recall': 0.8190045248868778,
 'sensitivity': 0.8190045248868778,
 'specificity': 0.8}
